### 基礎設定

In [ ]:
# 使用 LangChain 1.0 串接本地 大模型
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="qwen3:latest",
    temperature=0.5,
)

result = llm.invoke("What is the capital of the moon?")
print(result.content)

### Streaming

In [ ]:
from ollama import chat

prompt = "请写一句关于『时间』的繁体中文，严格要求『8 到 15 个字』（不含标点），且文中绝对不能出现『时、间、流、逝、光、阴』这六个字。"

stream = chat(
  model='qwen3:latest',
  messages=[{'role': 'user', 'content': prompt}],
  stream=True,
  think=False
)

content = ''
for chunk in stream:
  if chunk.message.content:
    print(chunk.message.content, end='', flush=True)
    # accumulate the partial content
    content += chunk.message.content

In [ ]:
# TODO: 使用 LangChain 1.0 實現

### Thinking / Reasoning

In [ ]:
from ollama import chat

prompt = "请写一句关于『时间』的繁体中文，严格要求『8 到 15 个字』（不含标点），且文中绝对不能出现『时、间、流、逝、光、阴』这六个字。"

stream = chat(
  model='qwen3:latest',
  messages=[{'role': 'user', 'content': prompt}],
  stream=True,
  think=True
)

in_thinking = False

for chunk in stream:
  if chunk.message.thinking and not in_thinking:
    in_thinking = True
    print('Thinking:\n', end='')

  if chunk.message.thinking:
    print(chunk.message.thinking, end='')
  elif chunk.message.content:
    if in_thinking:
      print('\n\nAnswer:\n', end='')
      in_thinking = False
    print(chunk.message.content, end='')

In [ ]:
# TODO: 使用 LangChain 1.0 實現

In [ ]:
# TODO: 使用 LangChain 1.0 實現
# TODO: 補充 Reasoning Streaming 功能

### Structured Output

In [ ]:
from ollama import chat
from pydantic import BaseModel

class Pet(BaseModel):
  name: str
  animal: str
  age: int
  color: str | None
  favorite_toy: str | None

class PetList(BaseModel):
  pets: list[Pet]

story = """
3歲的枯草色水豚阿呆正處於入定狀態，頭頂完美平衡著他最愛的玩具——一顆橘子。

這份寧靜激怒了7歲的米白吉娃娃毀滅者。他發出尖銳的戰吼，把阿呆的後腿當成他最愛的玩具——別人的腳後跟，衝上去狠狠咬了一口。

阿呆皮太厚完全沒感覺，只是緩慢地轉頭打了個哈欠。這微小的震動讓橘子滑落，精準地塞住了毀滅者正張大嘴狂吠的喉嚨。

世界瞬間安靜了。
"""

response = chat(
  model='qwen3:latest',
  messages=[{'role': 'user', 'content': story}],
  format=PetList.model_json_schema(),
)

pets = PetList.model_validate_json(response.message.content)
print(pets)

In [ ]:
# TODO: 使用 LangChain 1.0 實現

### Tool Calling

In [ ]:
from ollama import chat 

# Tools
def get_temperature(city: str) -> str:
  """Get the current temperature for a city
  
  Args:
    city: The name of the city

  Returns:
    The current temperature for the city
  """
  temperatures = {
    'New York': '22°C',
    'London': '15°C',
  }
  return temperatures.get(city, 'Unknown')

# User Message
messages = [{'role': 'user', 'content': "New York 溫度多少?"}]

# Main Loop
while True:
  stream = chat(
    model='qwen3:latest', # deepseek-r1:latest tool not supported issue
    messages=messages,
    tools=[get_temperature],
    stream=True,
    think=True,
  )

  thinking = ''
  content = ''
  tool_calls = []

  in_thinking = False
  in_content = False
  in_tool_calls = False
  # accumulate the partial fields
  for chunk in stream:

    # Thinking Part, In langchain it's an AIMessage
    if chunk.message.thinking:
      if not in_thinking:
        in_thinking, in_content, in_tool_calls = True, False, False
        print('\nThinking:\n', end='')
      thinking += chunk.message.thinking
      print(chunk.message.thinking, end='', flush=True)
    
    # Content Part, In langchain it's an HumanMessage
    if chunk.message.content:
      if not in_content:
        in_thinking, in_content, in_tool_calls = False, True, False
        print('\nAnswer:\n', end='')
      content += chunk.message.content
      print(chunk.message.content, end='', flush=True)
    
    # Tool Calls Part, In langchain it's an ToolMessage
    if chunk.message.tool_calls:
      if not in_tool_calls:
        in_thinking, in_content, in_tool_calls = False, False, True
        print('\nTool Calls:\n', end='')
      tool_calls.extend(chunk.message.tool_calls)
      print(chunk.message.tool_calls)

  # append accumulated fields to the messages
  if thinking or content or tool_calls:
    messages.append({'role': 'assistant', 'thinking': thinking, 'content': content, 'tool_calls': tool_calls})

  if not tool_calls:
    break

  # Execute the tool calls
  for call in tool_calls:
    if call.function.name == 'get_temperature':
      result = get_temperature(**call.function.arguments)
    else:
      result = 'Unknown tool'
    messages.append({'role': 'tool', 'tool_name': call.function.name, 'content': result})

In [ ]:
# TODO: 使用 LangChain 1.0 實現
# TODO: Hint: 使用 create_agent API

### 練習

* 測試不同的本地大模型
* 多模態
* Embedding